In [1]:
import pandas as pd
import numpy as np
import random
df = pd.read_csv("snacks.csv")
# if df[df.Snack == "MazMaz"]["Value"].item() > 8:
#     print("salam")
    
upBound = 4
downBound = 3
maximumWeight = 10
population = []

In [2]:
class KnapsackItem:
    def __init__(self, key_, weight_, value_):
        self.key = key_
        self.value = value_
        self.weight = weight_

In [3]:
numberOfSolutions = 1000
for i in range(0, numberOfSolutions):
    numberOfItems = random.randint(downBound, upBound)
    itemsKey = sorted(random.sample(np.arange(len(df)).tolist(), numberOfItems))
    solution = []
    for j in range(0, len(df)):
        if j in itemsKey:
            randomWeight = abs(np.random.normal(df.iloc[j, 1] / 2, 2))
            value = (randomWeight / df.iloc[j, 1])*df.iloc[j, 2]
            solution.append(KnapsackItem(j, randomWeight, value))
        else:
            solution.append(KnapsackItem(j, 0, 0))   
    population.append(solution)


In [4]:
def getSolutionPoint(solution_):
    weightSum = 0
    valueSum = 0
    numberOfNonzeros = 0
    for item in solution_:
        weightSum += item.weight
        valueSum += item.value
        if item.weight != 0:
            numberOfNonzeros += 1
    if weightSum > maximumWeight:
        return  maximumWeight - weightSum
    # if numberOfNonzeros < downBound:
    #     return numberOfNonzeros - downBound
    # if  numberOfNonzeros > upBound:
    #     return upBound - numberOfNonzeros
    else:
        return valueSum

    

In [5]:
population = sorted(population, key = lambda x : getSolutionPoint(x), reverse=True)

def weighted_choice(items):
    weights = [1 / (i + 1) for i in range(len(items))]
    return random.choices(items, weights=weights, k=2)



In [6]:
def crossOver(solution1, solution2):
    crossOverPoint = random.randint(1, len(solution1) - 1)
    return solution1[0:crossOverPoint] + solution2[crossOverPoint:]
    # return solution1[0:crossOverPoint] + solution2[crossOverPoint:], solution2[0:crossOverPoint] + solution1[crossOverPoint:]


In [7]:
def mutate(solution_):
    numberOfNonZero = 0
    weightSum = 0
    for item in solution_:
        weightSum += item.weight
        if item.weight != 0:
            numberOfNonZero += 1
    if weightSum > maximumWeight and weightSum != 0:
        normalizationFactor = weightSum / maximumWeight
        for item in solution_:
            item.weight = item.weight / normalizationFactor
            item.value = item.value / normalizationFactor
    elif weightSum < maximumWeight and weightSum != 0:
        normalizationFactor = maximumWeight / weightSum
        for item in solution_:
            item.weight = item.weight * normalizationFactor
            item.value = item.value * normalizationFactor
    if numberOfNonZero < downBound:
        for i in range(0, downBound - numberOfNonZero):
            for item in solution_:
                if item.weight == 0:
                    item.weight = abs(np.random.normal(df.iloc[item.key, 1] / 2, 2))
                    item.value = (item.weight / df.iloc[item.key, 1]) * df.iloc[item.key, 2]
    elif numberOfNonZero > upBound:
        diffrence = numberOfNonZero - upBound
        while diffrence != 0:
            randomIndex = random.randint(0, len(df) - 1)
            if solution_[randomIndex].weight != 0:
                solution_[randomIndex].weight = 0
                solution_[randomIndex].value = 0
                diffrence -= 1
    return solution_            




In [8]:
numberOfGenerations = 400
for i in range(0, numberOfGenerations):
    newPopulation = []
    for j in range(0, numberOfSolutions):
        parent1, parent2 = weighted_choice(population)
        newChild = crossOver(parent1, parent2)
        newChild = mutate(newChild)
        newPopulation.append(newChild)
    population = newPopulation
    sorted(population, key = lambda x : getSolutionPoint(x), reverse=True)    

        
        


In [9]:
for j in range(0, 1000):
    weightSum = 0
    valueSum = 0
    for item in population[j]:
        weightSum += item.weight
        valueSum += item.value
    if weightSum <= maximumWeight and valueSum >= 10:
        for item in population[j]:
            if item.weight != 0:
                print(df.iloc[item.key, 0], item.weight, item.value)
        print("------")          

# for item in population[0]:
#     print(df.iloc[item.key, 0], item.weight, item.value)

MazMaz 3.1998413514708752 3.1998413514708752
Jooj 1.9783171218394373 4.239250975370185
Hot-Dog 4.8218415266896875 3.6163811450172503
------
MazMaz 3.1998413514708752 3.1998413514708752
Jooj 1.9783171218394373 4.239250975370185
Hot-Dog 4.8218415266896875 3.6163811450172503
------
MazMaz 3.1998413514708752 3.1998413514708752
Jooj 1.9783171218394373 4.239250975370185
Hot-Dog 4.8218415266896875 3.6163811450172503
------
MazMaz 3.1998413514708752 3.1998413514708752
Jooj 1.9783171218394373 4.239250975370185
Hot-Dog 4.8218415266896875 3.6163811450172503
------
MazMaz 3.1998413514708752 3.1998413514708752
Jooj 1.9783171218394373 4.239250975370185
Hot-Dog 4.8218415266896875 3.6163811450172503
------
MazMaz 3.1998413514708752 3.1998413514708752
Jooj 1.9783171218394373 4.239250975370185
Hot-Dog 4.8218415266896875 3.6163811450172503
------
MazMaz 3.1998413514708752 3.1998413514708752
Jooj 1.9783171218394373 4.239250975370185
Hot-Dog 4.8218415266896875 3.6163811450172503
------
MazMaz 3.19984135147